# 1. Get miRNAs that are in (+) strand

In [9]:
# get all mRNA gene names from gencode.gtf
!grep -i miRNA ../binfo1-datapack1/gencode.gtf | awk '$3=="gene" && $7=="+" {print $10, $1, $4, $5}' > mirna_dm.txt

miRNA 중 (+) strand인 entity 들만 선정

In [2]:
import pandas as pd
import re
from collections import Counter
from scipy.stats import entropy
from tqdm import tqdm
from datetime import datetime
import pickle

model_name = 'get_Lin28a_binding_seqs'
version = 1
logger_file = f'log/{model_name}_v{version}'


def logger(text, show=True):
    text = f'[{datetime.now().replace(microsecond=0)}] {text}'
    with open(f'{logger_file}.log', 'a') as f:
        f.write(text+'\n')
    if show:
        print(text)

In [3]:
mrnas = pd.read_csv('mirna_dm.txt', delimiter=' ', header=None)
mrnas.columns = ['gene_id','chr','start','end']
mrnas

,gene_id,chr,start,end
0,ENSMUSG00002075967.1;,chr1,8886362,8886471
1,ENSMUSG00000098555.3;,chr1,12496210,12496330
2,ENSMUSG00002076346.1;,chr1,13332422,13332503
3,ENSMUSG00002075819.1;,chr1,14252802,14252901
4,ENSMUSG00000065559.3;,chr1,20749234,20749306
...,...,...,...,...
1170,ENSMUSG00002075445.1;,chrX,165740095,165740222
1171,ENSMUSG00002075850.1;,chrY,5542752,5542861
1172,ENSMUSG00002074919.1;,chrY,20134948,20135057
1173,ENSMUSG00002075449.1;,chrY,40639526,40639635


In [4]:
mrnas['gene_id'] = mrnas['gene_id'].str.split('.',expand=True)
mrnas # all gene ids are unique

,gene_id,chr,start,end
0,ENSMUSG00002075967,chr1,8886362,8886471
1,ENSMUSG00000098555,chr1,12496210,12496330
2,ENSMUSG00002076346,chr1,13332422,13332503
3,ENSMUSG00002075819,chr1,14252802,14252901
4,ENSMUSG00000065559,chr1,20749234,20749306
...,...,...,...,...
1170,ENSMUSG00002075445,chrX,165740095,165740222
1171,ENSMUSG00002075850,chrY,5542752,5542861
1172,ENSMUSG00002074919,chrY,20134948,20135057
1173,ENSMUSG00002075449,chrY,40639526,40639635


In [5]:
def count_base(seq):
    counts = Counter(seq.upper())
    counts = [counts['A'], counts['T'], counts['G'], counts['C']]
    index = counts.index(max(counts))
    return counts, index

In [12]:
threshold_entropy = 0.8
genes = []
seqs = []


for i in tqdm(range(len(mrnas))):
    gene = mrnas.iloc[i]
    chr = gene['chr']
    start = gene['start']
    end = gene['end']
    gid = gene['gene_id']
    
    query = f'{chr}:{start}-{end}'
    file = f'gene.pileup'

    !samtools view -b -o - ../binfo1-datapack1/CLIP-35L33G.bam $query | samtools mpileup - > $file
    pileup = pd.read_csv(file, sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
    pileup = pileup[(pileup['pos']>=start)&(pileup['pos']<=end)]
    if i==0:
        print(pileup.head())

    toremove = re.compile('[<>$*#^]')
    pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))
    pileup[['chrom', 'pos', 'matches']]

    max_entropy = 0
    max_pos = 0
    tot_pos = []
    tot_seq = []
    for pos in pileup['pos']:
        seq = pileup[pileup['pos'] == pos].iloc[0]['matches']

        # if seq is null : insert entropy as 1
        if seq != '' : 
            count_list, index = count_base(seq)
            entrop = entropy(count_list)
            tot_seq.append(index)
            tot_pos.append(pos)
            #print(pos, entrop, count_base(seq))

            if max_entropy < entrop:
                max_entropy = entrop
                max_pos = pos
                
    logger(f'{gid}, {max_entropy}', False)
    if max_entropy > threshold_entropy : # generate sequences
        tot_seq[tot_pos.index(max_pos)]=2 # attatched location = G(2)
        try:
            seq_st = tot_pos.index(max_pos-8)
            seq_end = tot_pos.index(max_pos+9)
            seq18 = tot_seq[seq_st:seq_end+1]
            genes.append(gid)
            seqs.append(seq18)   
            logger(f'> [{max_entropy}] {gid} is added!')
        except: 
            try:
                seq_st = tot_pos.index(max_pos-8)
                seq18 = tot_seq[seq_st:]
                genes.append(gid)
                seqs.append(seq18)   
                logger(f'> [{max_entropy}] {gid} is added!')
            except:
                try:
                    seq_end = tot_pos.index(max_pos+9)
                    seq18 = tot_seq[:seq_end+1]
                    genes.append(gid)
                    seqs.append(seq18)   
                    logger(f'> [{max_entropy}] {gid} is added!')
                except:
                    pass

  0%|          | 0/1175 [00:00<?, ?it/s]

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


  0%|          | 1/1175 [00:00<05:13,  3.74it/s]

   chrom      pos _ref  count  \
15  chr1  8886362    N    309   
16  chr1  8886363    N    309   
17  chr1  8886364    N    309   
18  chr1  8886365    N    308   
19  chr1  8886366    N    308   

                                            basereads  \
15  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
16  GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...   
17  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...   
18  CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...   
19  TTTTTT$TTTTTTTTTTT$T$TT$TTTTTTTTTTTTTTTTTTTTTT...   

                                                quals  
15  IIG9IHIIEI?IIIHHIHHBII7IIIBIGIGDH;HHGIHFEGIIFH...  
16  IIG?IHIIEI<IIIHGIHDBIG:HHG?F<B@GB8GHGGHGHGIIEH...  
17  IHD=IHFI@G@IHFIHIGH;II1III<IEFDAH3HH7HIGHIIIFH...  
18  IID=IHII@I@IIIIHIHH7II.III<IEID@H:FHGIIGIIIIHH...  
19  II@BGGIIEI:GHIIHHHHBII;IIIBHGIGEH9HHEIIGIGIIFH...  
[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


  0%|          | 2/1175 [00:00<04:04,  4.79it/s]

...

100%|█████████▉| 1173/1175 [05:10<00:00,  4.49it/s]

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


100%|█████████▉| 1174/1175 [05:10<00:00,  4.11it/s]

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


100%|██████████| 1175/1175 [05:10<00:00,  3.78it/s]


In [13]:
with open('seqs.pkl', 'wb') as f:
    pickle.dump((genes, seqs), f)
    
with open('seqs.pkl', 'rb') as f:
    a,b = pickle.load(f)
    
len(a)

20

In [14]:
genes

['ENSMUSG00000092827',
 'ENSMUSG00002075180',
 'ENSMUSG00002076697',
 'ENSMUSG00002075410',
 'ENSMUSG00000119232',
 'ENSMUSG00002075188',
 'ENSMUSG00002075547',
 'ENSMUSG00002075625',
 'ENSMUSG00000078035',
 'ENSMUSG00000065440',
 'ENSMUSG00000072837',
 'ENSMUSG00000092998',
 'ENSMUSG00000076272',
 'ENSMUSG00002075058',
 'ENSMUSG00002075904',
 'ENSMUSG00002075428',
 'ENSMUSG00002075079',
 'ENSMUSG00002076173',
 'ENSMUSG00000098269',
 'ENSMUSG00000065536']

In [15]:
seqs

[[1, 2, 2, 2, 1, 3, 1, 2, 2, 1, 1, 2, 2, 2, 3, 3, 3, 2],
 [0, 2, 2, 2, 3, 3, 1, 2, 2, 0, 1],
 [2, 3, 3, 0, 2, 3, 1, 2, 2, 0, 2, 2, 1, 2, 2],
 [0, 1, 3, 3, 0, 1, 2, 0, 2, 0, 3, 3, 0, 3, 1, 1, 1, 3],
 [2, 2, 0, 1, 3, 1, 3, 1, 2, 2, 2, 3, 2, 0, 2, 1, 0, 1],
 [2, 2, 1, 3, 3, 2, 2, 1, 2, 3, 2, 2, 0, 2, 0, 2, 3, 3],
 [1, 0, 1, 2, 2, 3, 3, 2, 2, 3, 1, 2, 2, 2, 3, 3, 0, 1],
 [3, 3, 3, 2, 2, 2, 0, 2, 3, 2, 2, 2, 2, 2, 0],
 [0, 1, 3, 1, 2, 1, 2, 2, 2, 0, 3, 1, 3, 0, 0, 0, 3, 1],
 [3, 3, 2, 2, 1, 0, 3, 0, 2, 2, 0, 2, 0, 1, 0, 0, 3, 1],
 [1, 1, 1, 1, 0, 2, 0, 0, 2, 3, 0, 0, 0, 2, 1, 2, 0, 1],
 [1, 2, 2, 1, 2, 3, 1, 3, 2, 2, 2, 0, 2, 1, 1, 3, 1, 3],
 [0, 2, 3, 1, 2, 1, 2, 0, 2, 0, 2, 3, 3, 3, 3, 3, 1, 0],
 [3, 1, 1, 2, 2, 1, 3, 1, 2, 2, 0, 2, 0, 0, 3, 0, 3, 1],
 [2, 3, 0, 2, 2, 0, 3, 2, 2, 3, 2, 3, 3, 3, 1, 2, 3, 2],
 [2, 1, 1, 0, 1, 3, 1, 3, 2, 1, 0, 0, 0, 3, 0, 2, 2, 2],
 [1, 2, 3, 3, 1, 2, 2, 2, 2, 0, 2, 0, 1, 2, 0, 1, 1, 0],
 [2, 2, 1, 3, 3, 2, 2, 1, 2, 3, 2, 2, 0, 2, 0, 2, 3, 3],
 [0, 2, 1, 0,